In [1]:
import pandas as pd
import numpy as np
import requests as rq
import time
from requests.exceptions import HTTPError
import json
from jsonschema import validate
from riotwatcher import LolWatcher, ApiError

In [3]:
API_KEY = "RGAPI-f5ffc1a2-3502-459e-9997-d9441d21f3d8"
base_diamond = "https://br1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/"
base_challenger = "https://br1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5/"
base_master = "https://br1.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5"
base_grand_master = "https://br1.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5"

base_summoner_infos = "https://br1.api.riotgames.com/lol/summoner/v4/summoners/by-name/"

In [4]:
watcher = LolWatcher(API_KEY)
my_region = 'br1'

In [4]:
def makeRequest(string):
    try:
        response = rq.get(string)
        response.raise_for_status()
    except HTTPError:
        if response.status_code == 429:
            time.sleep(120)
            makeRequest(string)
    return response.json()

In [5]:
def getLowEloInfos(json):
    df = pd.DataFrame(json)
    df["tier"] = df['tier'] + df["rank"]
    df.drop(["leagueId","queueType","rank","miniSeries"], axis=1,inplace=True)
    return df   

In [6]:
def getHighEloInfos(json):
    df_aux = pd.DataFrame(json["entries"])
    df_aux.drop("rank", axis=1, inplace=True)
    df_aux["tier"] = "MASTER"
    df_aux = df_aux.reindex(
        columns=['tier', 'summonerId', 'summonerName', 
                 'leaguePoints', 'wins', 'losses','veteran',
                 'inactive', 'freshBlood', 'hotStreak'])
    return df_aux

In [10]:
%%time
divisions = ["I","II","III","IV"]
tiers = ["DIAMOND","MASTER","GRANDMASTER","CHALLENGER"]
df = pd.DataFrame()
for i in tiers:
    if i == "DIAMOND":
        for j in divisions:
            string = base_diamond + i + "/" + j + "?api_key=" + API_KEY
            df_aux = getLowEloInfos(makeRequest(string))
            df = df.append(df_aux,ignore_index = True)
    
    elif i == "MASTER":
        string = base_master + "?api_key=" + API_KEY
        df_aux = getHighEloInfos(makeRequest(string))
        df = df.append(df_aux,ignore_index = True)
    
    
    elif i == "GRANDMASTER":
        string = base_grand_master + "?api_key=" + API_KEY
        df_aux = getHighEloInfos(makeRequest(string))
        df = df.append(df_aux,ignore_index = True)
    
    else:
        string = base_challenger + "?api_key=" + API_KEY
        df_aux = getHighEloInfos(makeRequest(string))
        df = df.append(df_aux,ignore_index = True)
        

CPU times: user 284 ms, sys: 51 ms, total: 335 ms
Wall time: 6.37 s


In [11]:
df.to_csv("data/usersBasicInfos.csv",index=False)

In [15]:
df = pd.read_csv("data/usersBasicInfos.csv")

In [16]:
df.shape

(2737, 10)

In [17]:
df.head(2)

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,DIAMONDI,a6XNnr-OC1673HAJaeCorjI1T9Sbe7gYJIBpbsu_CllJIOI,LokiFc,39,192,184,False,False,False,False
1,DIAMONDI,h_TlijF-s3RAtZaCyUVvBvgO-GGVuG60TqH3h7cTGrOKT18,lepanta,96,123,102,False,False,False,False


#### Esses dados serão usados como base. Tipo uma semente para a construção do data set final

#### Agora vamos enriquecer esses dados com mais informações 

In [11]:
def checkFields(response, fields):
    for i in fields:
        if i not in response:
            return False
    return True

In [12]:
def getSummonerInfos(summonerNames):
    list_dicts = []
    count = 0
    for i in summonerNames:
        count+=1
        if (count % 100) == 0:
            df_temp = pd.DataFrame(list_dicts)
            df_temp.to_csv("DataSummornerTemp.csv")
        string = base_summoner_infos + i + "?api_key=" + API_KEY
        response = makeRequest(string)
        infos_dict = {}
        if checkFields(response,["accountId","summonerLevel","puuid"]):
            infos_dict["summonerName"] = i
            infos_dict["accountId"] = response["accountId"]
            infos_dict["summonerLevel"] = response["summonerLevel"]
            infos_dict["puuid"] = response["puuid"]
            list_dicts.append(infos_dict)
    return list_dicts

In [18]:
names = df.summonerName.to_list()

In [19]:
len(names)

2737

In [22]:
%%time
infos_plus = getSummonerInfos(names)

CPU times: user 1min 18s, sys: 5.62 s, total: 1min 24s
Wall time: 1h 26min 47s


In [23]:
len(infos_plus)

2661

In [24]:
df_summoner = pd.DataFrame(infos_plus)

In [26]:
df_summoner.to_csv("data/summonerInfosPlus.csv",index=False)

In [27]:
df_summoner = pd.read_csv("data/summonerInfosPlus.csv",index_col="summonerName")

In [28]:
df_summoner.head(2)

,accountId,summonerLevel,puuid
summonerName,,,
LokiFc,bk1m2_GS-2L9loYOKil6EbsEHnQOCY15LRVByLuQvjf46Cc,291,AvAT9pAFILqL0Eo0kWmFOXug4KyIBh4QprnfHU4Ao2e3Bj...
lepanta,B6nuFHgma2xw3B4ZYsSQeb38pgQxgajWykeB8OEZHm0sSO8,131,z2NP0f9IPVNevXHBEiZ5wjOHHwmqifJM0St3OpTjgI8xon...


In [29]:
df_summonerFinal = df.join(df_summoner,how="inner",on="summonerName")

In [30]:
df_summonerFinal.columns

Index(['tier', 'summonerId', 'summonerName', 'leaguePoints', 'wins', 'losses',
       'veteran', 'inactive', 'freshBlood', 'hotStreak', 'accountId',
       'summonerLevel', 'puuid'],
      dtype='object')

In [31]:
df_summonerFinal.to_csv("data/SummonerFinalData.csv")

   ### Matchs Players

In [32]:
df_summoners = pd.read_csv("data/SummonerFinalData.csv")

In [33]:
summoners_id = df_summoners.accountId.to_list()

In [34]:
len(summoners_id)

2661

In [36]:
def getSummonerMatches(accountId):
    result = watcher.match.matchlist_by_account(my_region,accountId)["matches"]
    return result
    

In [37]:
%%time
summoner_list = []
for summoner in summoners_id:
    matches = {}
    matches["accountId"] = summoner
    try:
        matches["listMatches"] = getSummonerMatches(summoner)
    except:
        time.sleep(5)
        matches["listMatches"] = getSummonerMatches(summoner)     
    summoner_list.append(matches)  

CPU times: user 1min 27s, sys: 7.87 s, total: 1min 35s
Wall time: 53min 12s


In [40]:
df_matches = pd.DataFrame(summoner_list)

In [39]:
df_matches.to_csv("data/matchesBasicInfos.csv")

In [41]:
df_all = []

In [42]:
for i in df_matches.values:
    accountId = i[0]
    df_temp = pd.DataFrame(i[1])
    df_temp["accountId"] = accountId
    df_all.append(df_temp)

In [43]:
df_final = pd.concat(df_all)


In [47]:
df_final.to_csv("data/matchByAccounbtId.csv",index=None)

In [4]:
df_final = pd.read_csv("data/matchByAccounbtId.csv")

In [5]:
df_final.shape

(265905, 9)

In [6]:
df_final.head(3)

,platformId,gameId,champion,queue,season,timestamp,role,lane,accountId
0,BR1,2071507283,114,440,13,1601166961222,SOLO,TOP,bk1m2_GS-2L9loYOKil6EbsEHnQOCY15LRVByLuQvjf46Cc
1,BR1,2071513192,236,440,13,1601164763566,DUO,TOP,bk1m2_GS-2L9loYOKil6EbsEHnQOCY15LRVByLuQvjf46Cc
2,BR1,2071489057,518,440,13,1601162891562,SOLO,MID,bk1m2_GS-2L9loYOKil6EbsEHnQOCY15LRVByLuQvjf46Cc


In [7]:
a = df_final.groupby(['accountId'])['gameId'].apply(lambda grp: grp.nlargest(10)).values

In [8]:
df_final_filtered = df_final.query('gameId in @a')

In [9]:
df_final_filtered.to_csv('data/matchByAccounbtIdFiltered10.csv',index=False)

In [2]:
def getMatchStats(gameId):
    match_detail = watcher.match.by_id(my_region, gameId)
    participants = []
    gameStats = []
    for row in match_detail['participants']:
        participants_row = {}
        participants_row['gameId'] = gameId
        participants_row['champion'] = row['championId']
        participants_row['spell1'] = row['spell1Id']
        participants_row['spell2'] = row['spell2Id']
        participants_row['win'] = row['stats']['win']
        participants_row['kills'] = row['stats']['kills']
        participants_row['deaths'] = row['stats']['deaths']
        participants_row['assists'] = row['stats']['assists']
        participants_row['totalDamageDealt'] = row['stats']['totalDamageDealt']
        participants_row['goldEarned'] = row['stats']['goldEarned']
        participants_row['champLevel'] = row['stats']['champLevel']
        participants_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
        participants_row['item0'] = row['stats']['item0']
        participants_row['item1'] = row['stats']['item1']
        participants.append(participants_row)
    for row in match_detail['teams']:
        row['gameId'] = gameId
        gameStats.append(row)
    return pd.DataFrame(participants), pd.DataFrame(gameStats)

In [9]:
len(df_final_filtered.gameId.to_list())

53720

In [18]:
df_final = pd.read_csv('data/dataMatchesStatsBackup.csv')

In [21]:
gameId_used = df_final.gameId.to_list()

In [22]:
len(gameId_used)

46002

In [23]:
df_final_filtered = df_final_filtered.query( 'gameId not in @gameId_used')

In [24]:
len(df_final_filtered.gameId.to_list())

7340

 ### Matche Stats

In [28]:
%%time
dataframes_players_stats = []
gameStats = []
count = 0
for i in df_final_filtered.gameId.to_list():
    try:
        players , games = getMatchStats(i)
        dataframes_players_stats.append(players)
        gameStats.append(games) 
        if (count % 1000) == 0:
            print(count)
            pd.concat(gameStats).to_csv('data/dataMatchesStatsBackup.csv')
            pd.concat(dataframes_players_stats).to_csv('data/playersMatchesStatsBackup.csv')
        count+=1
    except ApiError as err:
        if err.response.status_code == 429:
            print('We should retry in {} seconds.'.format(err.headers['Retry-After']))
            print('this retry-after is handled by default by the RiotWatcher library')
            print('future requests wait until the retry-after time passes')
            time.sleep(err.headers['Retry-After'])
        else:
            print(err.response.status_code)
            pd.concat(gameStats).to_csv('data/dataMatchesStatsBackup.csv')
            pd.concat(dataframes_players_stats).to_csv('data/playersMatchesStatsBackup.csv')
            continue
            
            

0
504
504
1000
504
504
504
2000
504
504
504
3000
504
4000
504
5000
504
504
6000
504
7000
503
CPU times: user 4min 30s, sys: 16.4 s, total: 4min 46s
Wall time: 2h 27min 40s


#### Os passos abaixo só devem ser executados caso haja erro no processo acima

In [40]:
df1 = pd.read_csv('dataMatchesStatsBackup.csv')
df2 = pd.read_csv('playersMatchesStatsBackup.csv')

In [41]:
df1.drop('Unnamed: 0', inplace=True,axis=1)
df2.drop('Unnamed: 0', inplace=True,axis=1)

In [33]:
games = pd.concat(gameStats)
players = pd.concat(dataframes_players_stats)

In [44]:
players_final = pd.concat([players,df2])
games_final = pd.concat([games,df1])

In [45]:
games_final.to_csv('data/dataMatchesStats.csv', index=False)
players_final.to_csv('data/playersMatchesStats.csv',index=False)